<a href="https://colab.research.google.com/github/OnePassio/CE888/blob/master/Project/Experience_2_Detect_Covariate_Shift_in_microsoft_malware_prediction_200k_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Library

In [0]:
# Initial Library
# data analysis
import pandas as pd
import numpy as np
import random as rand

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# ignore warning message
import warnings
warnings.filterwarnings('ignore')

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 1.Load Data from Kaggle
Source Data: https://www.kaggle.com/c/microsoft-malware-prediction/

!kaggle competitions download -c microsoft-malware-prediction

In [2]:
# load data from kaggle
import os
os.environ['KAGGLE_USERNAME'] = "thomastran89" # username from the json file
os.environ['KAGGLE_KEY'] = "6d1907f735446c9c560bb4371074fecd" # key from the json file
!kaggle competitions download -c microsoft-malware-prediction # api copied from kaggle
# Unzip the data
!sample_submission.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip
# Check files after unzip
!ls

 99% 665M/672M [00:53<00:01, 4.55MB/s]
100% 672M/672M [00:53<00:00, 13.1MB/s]
 96% 129M/134M [00:09<00:00, 10.9MB/s]
100% 134M/134M [00:09<00:00, 14.4MB/s]
 99% 761M/768M [00:36<00:00, 20.1MB/s]
100% 768M/768M [00:36<00:00, 22.2MB/s]
/bin/bash: sample_submission.csv.zip: command not found
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
sample_data		   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [141]:
# load partial data for test first
sample_size= 200000 # data too heavy and can not load by Google Colab -> Test on 1M first rows

# Load Train Data
df_train = pd.read_csv("train.csv",skiprows=0, nrows=sample_size)

# Load Test Data
df_test= pd.read_csv("test.csv",skiprows=0, nrows=sample_size)

df_train.head()
#df_test.head()


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,...,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,171,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2668.0,9124.0,4.0,5.0,...,HDD,299451.0,0,4096.0,Desktop,18.9,1440.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,93,1482.0,18.0,119.0,64,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,2668.0,91656.0,4.0,5.0,...,HDD,102385.0,0,4096.0,Notebook,13.9,1366.0,768.0,Mobile,NaN,1.000000e+00,10.0.17134.1,amd64,rs4_release,17134,1,Professional,PROFESSIONAL,IBSClean,8.0,31,UNKNOWN,0,OFFLINE,Retail,NaN,0.0,NOT_SET,NaN,628.0,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,86,153579.0,18.0,64.0,49,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,4909.0,317701.0,4.0,5.0,...,SSD,113907.0,0,4096.0,Desktop,21.5,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,UUPUpgrade,7.0,30,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,142.0,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,88,20710.0,NaN,117.0,115,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,ExistsNotSet,1.0,1.0,Desktop,Windows.Desktop,1443.0,275890.0,4.0,5.0,...,UNKNOWN,227116.0,0,4096.0,MiniTower,18.5,1366.0,768.0,Desktop,NaN,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,17.0,64,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,355.0,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417e

# 2. Preprocessing Data

## 2.1. Remove some unused features to build model of dataset
Some column field data have a unique ID each row or have too much distinct values(>50) should be remove, because it will make hard for build ML model

In [142]:
# Get All Accept Field from dataset (Ignore the field with too much string values) to build ML model
MAX_VALUE_ACCEPT=50
listAcceptField_Train=[]
for x in list(df_train):
    count=df_train[x].nunique()
    #print("{} has unique value is:{}".format(x,count))
    if(count<MAX_VALUE_ACCEPT):
        listAcceptField_Train.append(x)

listAcceptField_Test=listAcceptField_Train.copy()
listAcceptField_Test.remove('HasDetections')

print("Accept Fields from dataset to build Model is:")
print(listAcceptField_Train)
print(listAcceptField_Test)

Accept Fields from dataset to build Model is:
['ProductName', 'EngineVersion', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'OrganizationIdentifier', 'Platform', 'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'PuaMode', 'SMode', 'SmartScreen', 'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorClass', 'Census_PrimaryDiskTypeName', 'Census_HasOpticalDiskDrive', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_InternalBatteryType', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSBuildNumber', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier', 'Census_OSWUAutoUpdateOptionsName', 'Census_IsPortableOperatingSystem', 'Census_GenuineStateName', 'Census_ActivationChannel', 'Census

In [143]:
#cache
df_train_id=df_train['MachineIdentifier'].copy()
df_test_id=df_test['MachineIdentifier'].copy()
# Just keep acceptable features
df_train=df_train[listAcceptField_Train].copy();
df_test=df_test[listAcceptField_Test].copy();

df_test.head()

,ProductName,EngineVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,win8defender,1.1.15400.5,0,7.0,0,1.0,1.0,1,18.0,windows10,x64,10.0.0.0,15063,768,rs2,Home,1.0,0,NaN,NaN,NaN,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,NaN,SSD,0,Notebook,Mobile,NaN,amd64,rs2_release,15063,Core,CORE,Reset,37.0,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,1,NaN,0.0,0,0,0.0,0.0,7.0
1,win8defender,1.1.15400.4,0,7.0,0,1.0,1.0,1,NaN,windows10,x64,10.0.0.0,16299,768,rs3,Home,1.0,0,NaN,NaN,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,NaN,HDD,0,Notebook,Mobile,NaN,amd64,rs3_release_svc_escrow,16299,Core,CORE,UUPUpgrade,7.0,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,1,NaN,0.0,0,0,0.0,1.0,12.0
2,win8defender,1.1.15300.6,0,7.0,0,2.0,1.0,1,NaN,windows10,x64,10.0.0.0,14393,768,rs1,Home,1.0,0,NaN,NaN,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2.0,5.0,NaN,SSD,0,Notebook,Mobile,NaN,amd64,rs1_release,14393,CoreSingleLanguage,CORE_SINGLELANGUAGE,Other,35.0,UNKNOWN,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,1,NaN,0.0,0,0,0.0,1.0,11.0
3,win8defender,1.1.15400.5,0,7.0,0,2.0,1.0,1,NaN,windows10,x64,10.0.0.0,16299,768,rs3,Home,1.0,0,NaN,NaN,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2.0,5.0,NaN,HDD,0,Notebook,Mobile,li-i,amd64,rs3_release,16299,CoreSingleLanguage,CORE_SINGLELANGUAGE,Upgrade,26.0,Notify,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,0.0,1,0.0,0.0,0,0,0.0,0.0,10.0
4,win8defender,1.1.15400.4,0,7.0,0,1.0,1.0,1,18.0,windows10,x64,10.0.0.0,16299,768,rs3,Home,1.0,0,NaN,NaN,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,NaN,HDD,1,Notebook,Mobile,NaN,amd64,rs3_release,16299,CoreSingleLanguage,CORE_SINGLELANGUAGE,Update,29.0,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,1,NaN,0.0,0,0,0.0,1.0,3.0


### 2.2. Fill NaN missing value

In [144]:
## missing values on train.csv
for i in df_train.columns:
    if df_train[i].dtype == 'object':
      df_train[i] = df_train[i].fillna(df_train[i].mode().iloc[0])
    if (df_train[i].dtype == 'int' or df_train[i].dtype == 'float'):
      df_train[i] = df_train[i].fillna(np.mean(df_train[i]))

## missing values on test.csv
for i in df_test.columns:
    if df_test[i].dtype == 'object':
      df_test[i] = df_test[i].fillna(df_test[i].mode().iloc[0])
    if (df_test[i].dtype == 'int' or df_test[i].dtype == 'float'):
      df_test[i] = df_test[i].fillna(np.mean(df_test[i]))
      
df_test.head()

,ProductName,EngineVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,win8defender,1.1.15400.5,0,7.0,0,1.0,1.0,1,18.000000,windows10,x64,10.0.0.0,15063,768,rs2,Home,1.0,0,on,0.003183,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,mid,SSD,0,Notebook,Mobile,lion,amd64,rs2_release,15063,Core,CORE,Reset,37.0,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,OEM:DM,0.0,0.0,Retail,0.000338,1,0.0,0.0,0,0,0.0,0.0,7.0
1,win8defender,1.1.15400.4,0,7.0,0,1.0,1.0,1,24.903396,windows10,x64,10.0.0.0,16299,768,rs3,Home,1.0,0,on,0.003183,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,mid,HDD,0,Notebook,Mobile,lion,amd64,rs3_release_svc_escrow,16299,Core,CORE,UUPUpgrade,7.0,AutoInstallAndRebootAtMaintenanceTime,0,IS_GENUINE,Retail,0.0,0.0,Retail,0.000338,1,0.0,0.0,0,0,0.0,1.0,12.0
2,win8defender,1.1.15300.6,0,7.0,0,2.0,1.0,1,24.903396,windows10,x64,10.0.0.0,14393,768,rs1,Home,1.0,0,on,0.003183,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2.0,5.0,mid,SSD,0,Notebook,Mobile,lion,amd64,rs1_release,14393,CoreSingleLanguage,CORE_SINGLELANGUAGE,Other,35.0,UNKNOWN,0,IS_GENUINE,OEM:DM,0.0,0.0,Retail,0.000338,1,0.0,0.0,0,0,0.0,1.0,11.0
3,win8defender,1.1.15400.5,0,7.0,0,2.0,1.0,1,24.903396,windows10,x64,10.0.0.0,16299,768,rs3,Home,1.0,0,on,0.003183,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2.0,5.0,mid,HDD,0,Notebook,Mobile,li-i,amd64,rs3_release,16299,CoreSingleLanguage,CORE_SINGLELANGUAGE,Upgrade,26.0,Notify,0,IS_GENUINE,OEM:DM,0.0,0.0,Retail,0.000000,1,0.0,0.0,0,0,0.0,0.0,10.0
4,win8defender,1.1.15400.4,0,7.0,0,1.0,1.0,1,18.000000,windows10,x64,10.0.0.0,16299,768,rs3,Home,1.0,0,on,0.003183,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4.0,5.0,mid,HDD,1,Notebook,Mobile,lion,amd64,rs3_release,16299,CoreSingleLanguage,CORE_SINGLELANGUAGE,Update,29.0,UNKNOWN,0,IS_GENUINE,Retail,0.0,0.0,Retail,0.000338,1,0.0,0.0,0,0,0.0,1.0,3.0


## 2.3. label encoding

In [145]:
# encoder, alternative is convert to dummies
number = LabelEncoder()
for i in df_train.columns:
    if (df_train[i].dtype == 'object'):
      df_train[i] = number.fit_transform(df_train[i].astype('str'))
      df_train[i] = df_train[i].astype('object')

for i in df_test.columns:
    if (df_test[i].dtype == 'object'):
      df_test[i] = number.fit_transform(df_test[i].astype('str'))
      df_test[i] = df_test[i].astype('object')

df_test.head()

,ProductName,EngineVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,3,46,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,15063,768,2,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,4.0,5.0,2,1,0,16,3,15,0,1,15063,1,1,5,37.0,0,0,1,0,0.0,0.0,3,0.000338,1,0.0,0.0,0,0,0.0,0.0,7.0
1,3,45,0,7.0,0,1.0,1.0,1,24.903396,0,1,0,16299,768,3,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,4.0,5.0,2,0,0,16,3,15,0,3,16299,1,1,6,7.0,0,0,1,2,0.0,0.0,3,0.000338,1,0.0,0.0,0,0,0.0,1.0,12.0
2,3,43,0,7.0,0,2.0,1.0,1,24.903396,0,1,0,14393,768,1,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,2.0,5.0,2,1,0,16,3,15,0,0,14393,4,4,3,35.0,5,0,1,0,0.0,0.0,3,0.000338,1,0.0,0.0,0,0,0.0,1.0,11.0
3,3,46,0,7.0,0,2.0,1.0,1,24.903396,0,1,0,16299,768,3,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,2.0,5.0,2,0,0,16,3,13,0,2,16299,4,4,8,26.0,3,0,1,0,0.0,0.0,3,0.000000,1,0.0,0.0,0,0,0.0,0.0,10.0
4,3,45,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,16299,768,3,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,4.0,5.0,2,0,1,16,3,15,0,2,16299,4,4,7,29.0,5,0,1,2,0.0,0.0,3,0.000338,1,0.0,0.0,0,0,0.0,1.0,3.0


## 2.4. creating a new feature origin


In [146]:
df_train['origin'] = 0
df_test['origin'] = 1
df_test.head()

,ProductName,EngineVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,origin
0,3,46,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,15063,768,2,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,4.0,5.0,2,1,0,16,3,15,0,1,15063,1,1,5,37.0,0,0,1,0,0.0,0.0,3,0.000338,1,0.0,0.0,0,0,0.0,0.0,7.0,1
1,3,45,0,7.0,0,1.0,1.0,1,24.903396,0,1,0,16299,768,3,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,4.0,5.0,2,0,0,16,3,15,0,3,16299,1,1,6,7.0,0,0,1,2,0.0,0.0,3,0.000338,1,0.0,0.0,0,0,0.0,1.0,12.0,1
2,3,43,0,7.0,0,2.0,1.0,1,24.903396,0,1,0,14393,768,1,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,2.0,5.0,2,1,0,16,3,15,0,0,14393,4,4,3,35.0,5,0,1,0,0.0,0.0,3,0.000338,1,0.0,0.0,0,0,0.0,1.0,11.0,1
3,3,46,0,7.0,0,2.0,1.0,1,24.903396,0,1,0,16299,768,3,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,2.0,5.0,2,0,0,16,3,13,0,2,16299,4,4,8,26.0,3,0,1,0,0.0,0.0,3,0.000000,1,0.0,0.0,0,0,0.0,0.0,10.0,1
4,3,45,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,16299,768,3,4,1.0,0,1,0.003183,7,1.0,1.0,7,1,4.0,5.0,2,0,1,16,3,15,0,2,16299,4,4,7,29.0,5,0,1,2,0.0,0.0,3,0.000338,1,0.0,0.0,0,0,0.0,1.0,3.0,1


# 3. Detect Covariate Shift: Detect Dataset Shift feature

In [0]:
training = df_train.drop('HasDetections',axis=1) # droping target variable

## taking sample from training and test data
training = training.sample(50000, random_state=12)
testing = df_test.sample(50000, random_state=11)
## combining random samples
combi_data = training.append(testing)
y = combi_data['origin']
combi_data.drop('origin',axis=1,inplace=True)

In [148]:
## modelling
from sklearn.model_selection import cross_val_score

ROC_AUC_THRESHOLD=0.8
model = RandomForestClassifier(n_estimators = 100, max_depth = 5,min_samples_leaf = 5)
drop_list = []
print("Features have roc_auc greater than {} are:".format(ROC_AUC_THRESHOLD))
for i in combi_data.columns:
  score = cross_val_score(model,pd.DataFrame(combi_data[i]),y,cv=2,scoring='roc_auc')
  if (np.mean(score) > ROC_AUC_THRESHOLD):
    drop_list.append(i)
    print(i,np.mean(score))


Features have roc_auc greater than 0.8 are:
ProductName 0.9999638016000001
EngineVersion 0.8590666299999999
PuaMode 1.0
SMode 0.8793892628000001
Census_DeviceFamily 0.9992609492
Census_ChassisTypeName 0.9849957888
Census_InternalBatteryType 0.9993774188
Census_OSEdition 0.8101521796
Census_OSSkuName 0.812396394
Census_IsFlightingInternal 0.91572
Census_ThresholdOptIn 0.9464418232


In [149]:
print("Total Features of dataset is {}, dataset shift features are {}".format(len(list(combi_data)),len(drop_list)))
print(drop_list)
remain_features=[]
for x in list(combi_data):
  if(x not in drop_list):
    remain_features.append(x)
print("features have low drift value:")
print(remain_features)

Total Features of dataset is 56, dataset shift features are 11
['ProductName', 'EngineVersion', 'PuaMode', 'SMode', 'Census_DeviceFamily', 'Census_ChassisTypeName', 'Census_InternalBatteryType', 'Census_OSEdition', 'Census_OSSkuName', 'Census_IsFlightingInternal', 'Census_ThresholdOptIn']
features have low drift value:
['IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'OrganizationIdentifier', 'Platform', 'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'SmartScreen', 'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor', 'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorClass', 'Census_PrimaryDiskTypeName', 'Census_HasOpticalDiskDrive', 'Census_PowerPlatformRoleName', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSBuildNumber', 'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier', 'Census_OSWUAutoUpdateOptionsN

# 4. Build classifier prediction without features having a drift value greater

In [150]:

from sklearn.model_selection import train_test_split
#Get the values and run a classifier 
features= remain_features

# convert to dummies
X=df_train[features].copy()
y=df_train['HasDetections']

# 70% train, 30% test from dataset train.csv
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

X_train

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,OrganizationIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_HasOpticalDiskDrive,Census_PowerPlatformRoleName,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
129980,0,7.0,0,1.0,1.0,1,18.000000,0,2,0,17134,256,4,6,1.0,0,8,0.0,1.0,2,2.0,5.0,2,0,0,1,2,5,17134,4,39.0,2,0,1,2,0.0,3,0,0.0,0.0,0,0,0.0,0.0,7.0
28264,0,7.0,0,2.0,2.0,1,37.000000,0,1,0,17134,768,4,4,1.0,0,7,1.0,1.0,7,4.0,5.0,2,0,0,3,0,5,17134,6,9.0,2,0,1,2,0.0,3,1,0.0,0.0,0,0,0.0,0.0,15.0
69875,0,7.0,0,1.0,1.0,1,24.870919,0,2,0,14393,256,1,6,1.0,0,7,1.0,1.0,7,4.0,5.0,2,0,0,3,2,0,14393,7,8.0,3,0,1,2,0.0,3,0,0.0,0.0,0,0,0.0,0.0,11.0
191565,0,7.0,0,1.0,1.0,1,27.000000,0,1,0,17134,256,4,6,1.0,0,7,1.0,1.0,2,4.0,5.0,2,0,0,1,0,5,17134,6,27.0,2,0,0,2,0.0,3,0,0.0,0.0,0,0,0.0,1.0,15.0
138165,0,7.0,0,1.0,1.0,1,27.000000,0,1,0,16299,768,3,4,1.0,0,7,1.0,1.0,7,12.0,5.0,2,1,0,3,0,2,16299,3,5.0,5,0,1,0,0.0,3,1,0.0,0.0,0,0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146463,0,7.0,0,1.0,1.0,1,27.000000,0,1,0,16299,768,3,4,1.0,0,7,1.0,1.0,7,2.0,5.0,2,0,0,3,0,3,16299,2,27.0,5,0,0,2,0.0,3,0,0.0,0.0,0,0,0.0,0.0,15.0
135637,0,7.0,0,1.0,1.0,1,18.000000,0,1,0,10240,768,5,4,1.0,0,7,1.0,1.0,7,2.0,1.0,2,0,1,3,0,12,10240,2,8.0,5,0,0,2,0.0,1,0,0.0,0.0,0,0,0.0,0.0,1.0
175188,0,7.0,0,1.0,1.0,1,27.000000,0,2,0,16299,256,3,6,1.0,0,7,1.0,1.0,2,2.0,5.0,2,3,1,8,2,2,16299,6,9.0,3,0,1,2,0.0,3,0,0.0,0.0,0,0,0.0,1.0,10.0
74907,0,7.0,0,1.0,1.0,1,24.870919,0,1,0,16299,768,3,4,1.0,0,7,1.0,1.0,1,8.0,5.0,2,0,0,3,0,3,16299,5,8.0,5,0,1,0,0.0,3,1,0.0,0.0,1,1,0.0,1.0,1.0


In [151]:

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

# main Model
clf = ExtraTreesClassifier(n_estimators=1000, max_depth=4)
clf.fit(X_train,y_train)

# test with dummy model
#dummy_clf = DummyClassifier() # generates predictions by respecting the training set's class distribution
#dummy_clf.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=4, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [152]:
scores = cross_val_score(clf, X_train, y_train, cv=10,scoring=make_scorer(accuracy_score))
#dummy_scores = cross_val_score(dummy_clf, X_train, y_train, cv=10, scoring=make_scorer(accuracy_score))

#Report the results of 10-Kfold stratified cross-validation of 2 method
print("ACC: %0.2f +/- %0.2f" % (scores.mean(), scores.std()))
#print("Dummy ACC: %0.2f +/- %0.2f" % (dummy_scores.mean(), dummy_scores.std()))

ACC: 0.60 +/- 0.00


4.1 Test On test.csv

In [0]:
# Load Test Data all
#df_test = pd.read_csv("test.csv",skiprows=0)
features= remain_features
testFeatures=df_test[features].copy()
pred = clf.predict(testFeatures)
columns = ['HasDetections']
sub = pd.DataFrame(data=pred,columns=columns)
sub['MachineIdentifier'] = df_test_id
sub = sub[['MachineIdentifier','HasDetections']]
sub.to_csv('predict_without_drift_value_greater.csv', index=False)

In [0]:
#download result
from google.colab import files
files.download('predict_without_drift_value_greater.csv')

In [160]:
sub

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,1
1,00000176ac758d54827acd545b6315a5,1
2,0000019dcefc128c2d4387c1273dae1d,0
3,0000055553dc51b1295785415f1a224d,0
4,00000574cefffeca83ec8adf9285b2bf,1
...,...,...
199995,06847cc2f9d38f0a2b02f2e0f17e43db,1
199996,06847e7889f1328068c9cec8c88de6d3,0
199997,06847f805946628cc03d33f4092f5adf,1
199998,0684812676b7978f2a094abdf76334ee,1
